In [95]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [96]:
torono_df = pd.read_csv("./toronto_lat_lon.csv")
torono_df = torono_df.dropna().reset_index()
torono_df.head()

,index,Postcode,borough,neighbourhood,latitude,longitude
0,0,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.1930
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7878,-79.1564
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7678,-79.1866
3,3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [97]:
geolocator = Nominatim(user_agent='myapp')
location = geolocator.geocode('Toronto, CA')
latitude = location.latitude
longitude = location.longitude

In [98]:

toronto_map = folium.Map(location=[latitude,longitude],zoom_start=13)
toronto_map

In [99]:
for lat,lon, borough, neighbourhood in zip(torono_df['latitude'], torono_df['longitude'], torono_df['borough'], torono_df['neighbourhood']):
    #print("Borough - {}, Neighbourhood - {}, latitude - {}, longitude - {}".format(borough,neighbourhood,lat,lon))
    label = folium.Popup(neighbourhood)
    
    folium.CircleMarker(
        location=[lat,lon],
        color = 'red',
        radius = 5,
        popup=label,
        parse_html = True).add_to(toronto_map)
toronto_map